# Features
- 문서 semantic multi-indexing 처리와 저장
- ```TODO``` 패널데이터 인과추론 처리 Tool set 제공

In [447]:
import logging
import faiss
import numpy as np
import re
import polars as pl
import scipy
import uuid

from tqdm import tqdm
from typing import List, Optional

from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.chains import LLMChain
from langchain.document_loaders import JSONLoader
from langchain.retrievers import BM25Retriever, ContextualCompressionRetriever
from langchain.retrievers.document_compressors import DocumentCompressorPipeline, EmbeddingsFilter
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.utils.math import cosine_similarity
from langchain_community.vectorstores import FAISS
from langchain_core.documents.base import Document
from langchain_core.runnables import RunnableBranch, RunnableLambda, RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [448]:
update_vector_index = {
    "chunk": False, 
    "expected_query": False, 
    "intent_summary": False, 
}

max_tokens = 1024

In [449]:
docs_path = "/mnt/d/temp/user/ed/mart/mmplastic/20240725_154000_000000/data.json"
chunk_vector_index_dirpath = "/mnt/d/temp/user/ed/warehouse/mmplastic/vector_index/chunk_vectors.faiss/20240807_100000_000000"
summary_vector_index_dirpath = "/mnt/d/temp/user/ed/warehouse/mmplastic/vector_index/summary_vectors.faiss/20240807_110000_000000"
expected_query_vector_index_dirpath = "/mnt/d/temp/user/ed/warehouse/mmplastic/vector_index/expected_query_vectors.faiss/20240807_150000_000000"
intent_summary_vector_index_dirpath = "/mnt/d/temp/user/ed/warehouse/mmplastic/vector_index/intent_summary_vectors.faiss/20240808_120000_000000"

embedding_model_name = "gemma-2-embed"
lang_model_name = "tiger-gemma2"

question = "미세플라스틱이 햇빛에 노출되면 오염줄질을 흡수하는 연구와 관련된 미국 대학교를 찾고있어"


chunk_vector_db: Optional[FAISS] = None
expected_query_vector_db: Optional[FAISS] = None
intent_summary_vector_db: Optional[FAISS] = None


def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["title"] = record.get("title")
    metadata["date"] = record.get("date")
    return metadata

loader = JSONLoader(
    file_path=docs_path,
    jq_schema=".[]",
    content_key="content",
    text_content=True,
    metadata_func=metadata_func,
)
# https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/custom/

bm25_retriever = BM25Retriever.from_documents(loader.lazy_load(), preprocess_func=lambda x: re.split("[-\n,\s+]", x))

def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            # [f"{d.metadata['seq_num']}-{d.metadata['sub_seq_num']} RANK:{i+1}\n{d.metadata['title'][:20]}:\n\n" + d.page_content for i, d in enumerate(docs)]
            [f"RANK:{i+1}\nSEQ_NUM:[{d.metadata['seq_num']}]\n{d.metadata['title'][:64]}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )
def pretty_print_docs_with_similarity(docs):
    print(
        f"\n{'-' * 100}\n".join(
            # [f"{d.metadata['seq_num']}-{d.metadata['sub_seq_num']} RANK:{i+1}\n{d.metadata['title'][:20]}:\n\n" + d.page_content for i, d in enumerate(docs)]
            [f"RANK:{i+1}\nSEQ_NUM:[{d.metadata['seq_num']}], SIM[{d.state['query_similarity_score'] if 'query_similarity_score' in d.state else 'N/A'}]\n{d.metadata['title'][:64]}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

def remove_list_prefixes(input_string):
    lines = input_string.split('\n')
    cleaned_lines = []
    
    for line in lines:
        # Remove leading hyphens or numbered prefixes
        cleaned_line = re.sub(r'^\s*[-\d#]+\.\s*', '', line)
        cleaned_lines.append(cleaned_line)
    
    return '\n'.join(cleaned_lines)

## Splitters

In [450]:
text_chunk_splitter = RecursiveCharacterTextSplitter(
    separators="\n\n",
    chunk_size=200,
    chunk_overlap=10,
    keep_separator=True
)
overlapped_sentences_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=50,
    is_separator_regex=True,
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ]
)
sentence_splitter = RecursiveCharacterTextSplitter(
    # separators="\n\n",
    chunk_size=80,
    chunk_overlap=8,
    keep_separator=True,
    is_separator_regex=True,
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        # "",
    ]
)

## Model Declaration

In [451]:
embeddings = OllamaEmbeddings(
    model=embedding_model_name
)
llm = ChatOllama(
    model=lang_model_name,
    temperature=0.8,
    num_predict=max_tokens,
    num_gpu=-1
)

## Chain Definitions

### Input Chain

In [452]:
input_chain = {"question": RunnablePassthrough()}

### Quesion Classification
- SentenceType
- Intent
> SentenceType:서술형 Intent:기타 이면 외부 LLM으로 별도 처리?

#### SentenceType Classification Chain

In [453]:
sent_type_prompt = PromptTemplate.from_template(
    """아래 내용이 '질문형' 혹은 '서술형'인지 분류해줘. 분류만 짧게 답변하고, 분류기준은 아래를 참고해줘
    - 질문형: 무엇을 궁금해하는지가 텍스트에 들어있는 경우
    - 서술형: 질문이 아닌 서술하는 여러 개의 문장 형태를 갖고있는 경우

    내용: {question}
    분류:"""
)
sent_type_chain = (sent_type_prompt | llm | StrOutputParser())

#### Intent Classification Chain

In [454]:
INTENT = ["현상", "영향", "대응", "기타"]
intent_prompt = PromptTemplate.from_template(
    """사용자 질문을 '현상', '영향', '대응', '기타'로 분류해줘. 분류만 짧게 답변하고, 분류기준은 아래를 참고해줘
    - 현상: 미세플라스틱 자체를 자세히 설명하거나 자연이나 사회에서 나타난 경우
    - 영향: 미세플라스틱이 사람에게 건강이나 신처적으로 나쁜 영향을 주는 경우
    - 대응: 미세플라스틱 문제를 해결하기 위한 연구 결과나 발표 보고 등의 대응인 경우
    - 기타: 위 세가지에 해당하지 않는 경우

    질문: {question}
    분류:"""
)
def apply_intent(x):
    while result not in INTENT:
        result = llm.invoke(x["question"])
    return result

intent_chain = (
    intent_prompt | llm | StrOutputParser()
)

### Keyword Extraction Chain

In [455]:
keyword_extract_prompt = PromptTemplate(
    input_variables=["question"],
    template="""아래 질문에서 누가, 무엇을, 어떻게, 이유 등과 관련된 가장 중요한 키워드만 숫자없이 답변해줘. ',' 구분자로 답변해줘. 
    기본 질문: {question}
    """
)
keyword_extract_chain = keyword_extract_prompt | llm | StrOutputParser()

### Metadata Extraction Chain
- Question Classification
  - SentenceType / Intent Classification
- Keywords

In [456]:
preproc_chain = (
    RunnableParallel(
        intent=intent_chain,
        sentence_type=sent_type_chain,
        keywords=keyword_extract_chain
    )
)

In [457]:
question_classify_chain = (
    RunnableParallel(
        sentence_type=lambda x: sent_type_chain.invoke(x),
        intent=lambda x: intent_chain.invoke(x),
        question=lambda x: x["question"]
    )
)


### Expected Query Chain

In [458]:
# doc = "보통 미세플라스틱은 5㎜ 이하의 작은 플라스틱 조각을 말한다. 1㎛ 이하는 나노 플라스틱으로 분류된다. 인간이 만들어낸 물건에서 떨어져 나간 이 작은 플라스틱은 대기, 담수, 해양, 지하수, 토양 등 모든 곳에 흩뿌려져 지구 전체를 순환한다."
# num_q = int(len(doc) / 30)

expected_queries_prompt = PromptTemplate.from_template(
    """
    아래 문장에서 만들 수 있는 핵심질문 {num_q}가지 만들어줘. 문장 마침표를 제외한 특수기호는 모두 제거해줘. 
    {document}
    """
)

In [459]:
def estimate_num_queries(x):
    return {"num_q": int(len(x["document"]) / 20), "document": x["document"]}

def fetch_expected_queries(text):
    maybe_queries = remove_list_prefixes(text)
    queries = list(filter(lambda d: d, maybe_queries.split("\n")))
    # queries = list(filter(lambda d: d, maybe_queries))
    return {"expected_queries": queries}

expected_query_chain = (
    {"document": RunnablePassthrough()} |
    RunnableLambda(estimate_num_queries) | 
    {"document": lambda x: x["document"], "num_q": lambda x: x["num_q"]} | expected_queries_prompt | llm | StrOutputParser() |
    RunnableLambda(fetch_expected_queries) |
    RunnablePassthrough.assign(expected_queries=lambda x: x["expected_queries"])
    # JsonOutputParser()
)

### Split Chain

In [460]:
def split_docs(x):
    question = x["question"]
    splits = text_chunk_splitter.split_text(question)
    input_metadata = dict(x.items())
    del input_metadata["question"]
    return [
        {
            "id": str(uuid.uuid1()),
            "metadata": dict(
                **input_metadata, **{"chunk_seq_num": chunk_seq}
            ),
            "page_content": s,
            "question": question
        } for chunk_seq, s in enumerate(splits)
    ]
    # return [
    #     Document(
    #         page_content=s,
    #         id=str(uuid.uuid1()),
    #         metadata=dict(**input_metadata, **{"chunk_seq_num": i})
    #     ) for i, s in enumerate(splits)
    # ]

split_chain = (
    input_chain |
    question_classify_chain |
    RunnableParallel(
        {
            "question": lambda x: x["question"],
            "sentence_type": lambda x: x["sentence_type"],
            "intent": lambda x: x["intent"],
        }
    ) |
    RunnableLambda(split_docs) 
    # StrOutputParser()
)

split_chain.invoke(question)

[{'id': '1a37ec50-5602-11ef-983a-00155d610555',
  'metadata': {'sentence_type': '질문형', 'intent': '대응', 'chunk_seq_num': 0},
  'page_content': '미세플라스틱이 햇빛에 노출되면 오염줄질을 흡수하는 연구와 관련된 미국 대학교를 찾고있어',
  'question': '미세플라스틱이 햇빛에 노출되면 오염줄질을 흡수하는 연구와 관련된 미국 대학교를 찾고있어'}]

In [461]:
def preproc_batch(documents: List[dict]) -> List[dict]:
    metadata = preproc_chain.batch(documents)
    for doc, metadatum in zip(documents, metadata):
        doc.update(metadatum)
    return documents

split_chunk_chain = (
    {"seq_num": lambda x: x["seq_num"], "question": lambda x: x["question"]} |
    RunnableLambda(split_docs) |
    RunnableLambda(preproc_batch) 
)

# doc = """유엔환경계획(UNEP), 우려되는 화학물질 및 고분자(polymer) 사용을 전 세계적으로 금지하는 새로운 글로벌 조약 초안*을 발표함(9월 4일).\n\n일명 ‘제로 초안(zero draft)‘으로 알려진 이 조약은 UN 당사국이 플라스틱 오염을 줄이기 위한 방안에 관하여 작성된 최초의 문서로, 11월에 있을 3차 회담에 앞서 발표됨. UNEP는 2024년도까지 조약의 완성을 마무리하는 것을 목표로 함.\n\n조약에 명시된 요구사항은 국가 정책기관에 전달되어 궁극적으로 플라스틱을 생산 및 사용하는 기업과, 화학첨가제를 제조 및 공급하는 기업에 영향을 미칠 예정.\n\n조약 초안에 따르면, 조약 당사국은 미세플라스틱을 포함한 유해화학물질이 대기, 토양, 수질 및 생태계로 배출되는 것을 ‘방지 및 제거‘ 해야 한다고 명시하고 있으며, 이를 위해 아 래 3 가지 방안을 제시함\n\n- 방안 1 : 각 국가의 플라스틱 생산에 우려되는 화학물질 및 고분자 사용을 ‘금지 및 제거‘하도록 구속함\n\n- 방안 2 : 각 국가의 플라스틱 사용을 ‘최소화‘하도록 의무화 함\n\n위 두 가지 방안은 화학물질 및 고분자 생산자·수입자로 하여금 플라스틱 제품 lifecycle 에 걸쳐 건강 및 환경에 대한 유해성을 포함한 화학성분의 정보를 공개해야 함. 또한 화학물질을 추적할 수 있도록 적절한 조치를 취하고, 라벨링 요건을 수립해야 함.\n\n- 방안 3 : 가장 구속력이 낮은 방안으로, 화학물질 첨가 및 사용에 대하여 각 국가의 재량에 맡기고 개발된 전략을 공유하도록 함\n\n \n\n우려되는 화학물질 및 고분자 정의\n\n조약의 효력은 우려되는 화학물질 및 고분자의 정의에 따라 달라질 수 있으며, 다음과 같은 사항이 포함됨.\n\n- 발암성·돌연변이성·생식독성(CMR), 내분비장애(EDC), 잔류성·축척성·독성(PBT) 물질을 포함하여 인체 및 환경에 유해한 화학물질\n\n- 쉽게 재활용할 수 없는 고분자 및 브롬계난연제를 포함하여 안전하고 품질 높은 2차 재료의 재활용성 또는 순환성을 저해하는 물질\n\n- 미세플라스틱과 같이 환경에서 잘 분해되지 않아 방출했을 때 위험성이 있는 물질\n\n- 오존층 파괴 및 지구온난화(global warming potential, GWP) 물질\n\n- 고우려 고분자 물질\n\n- 플라스틱 제품으로부터 이동 및 방출 가능성 있는 물질\n\nNGO 단체는 우려되는 화학물질, 고분자 및 플라스틱 제품에 대한 규제 방안 합의에 있어 세부사항의 중요성을 당부함."""
# split_chunk_chain.invoke({"question": doc, "seq_num": 999})

### Intent-based Summarizing Chains

In [462]:
### For Indexing Pipeline
phenomenon_prompt = PromptTemplate.from_template("""
시간이나 장소를 포함한 현상을 대상과 수치를 함께 설명한 요약들로 30자 내로 짧게 정리해줘

텍스트: {question}
""", name="현상")
impact_prompt = PromptTemplate.from_template("""
사람이나 동물에게 어떤 나쁜 영향이 있는지 속성과 수치를 함께 설명한 요약들을 30자 내로 짧게 정리해줘

텍스트: {question}
""", name="영향")
response_prompt = PromptTemplate.from_template("""
대응을 하는 주체, 어떤 대응을 했는지에 대한 핵심요약을 30자 내로 짧게 정리해줘. 텍스트에 근거해서 답변해줘

텍스트: {question}
""", name="대응")
etc_prompt = PromptTemplate.from_template("""
핵심내용 위주로 30자 내로 짧게 요약 정리해줘

텍스트: {question}
""", name="기타")

### For Service Online
phenomenon_announce_prompt = PromptTemplate.from_template("""
시간이나 장소를 포함한 현상을 대상과 수치를 함께 설명한 요약들로 30자 내로 짧게 정리해줘
항상 "미세플라스틱 현상을 설명 드립니다."를 답변앞에 붙여줘

텍스트: {question}
""", name="현상")
impact_announce_prompt = PromptTemplate.from_template("""
사람이나 동물에게 어떤 나쁜 영향이 있는지 속성과 수치를 함께 설명한 요약들을 30자 내로 짧게 정리해줘
항상 "미세플라스틱이 자연에 미치는 영향을 전달 드립니다."를 답변앞에 붙여줘

텍스트: {question}
""", name="영향")
response_announce_prompt = PromptTemplate.from_template("""
대응을 하는 주체, 어떤 대응을 했는지에 대한 핵심요약을 30자 내로 짧게 정리해줘. 텍스트에 근거해서 답변해줘
항상 "미세플라스틱의 대응 방안을 알려 드립니다."를 답변앞에 붙여줘

텍스트: {question}
""", name="대응")
etc_announce_prompt = PromptTemplate.from_template("""
핵심내용 위주로 30자 내로 짧게 요약 정리해줘
항상 "미세플라스틱과 관련해 설명 드립니다."를 답변앞에 붙여줘

텍스트: {question}
""", name="기타")

phenomenon_chain = phenomenon_prompt | llm
impact_chain = impact_prompt | llm
response_chain = response_prompt | llm
etc_chain = etc_prompt | llm

In [463]:
### Intent-based Branching Chain

summarize_branch_chain = RunnableBranch(
    (
        lambda x: "현상" in x["intent"], phenomenon_chain
    ),
    (
        lambda x: "영향" in x["intent"], impact_chain
    ),
    (
        lambda x: "대응" in x["intent"], response_chain
    ),
    etc_chain
) | StrOutputParser()

In [464]:
def summarize(doc):
    summary = summarize_branch_chain.invoke(doc.page_content)
    doc.page_content = summary
    return doc

intent_summary_doc_chain = (

    RunnableParallel(
        page_content=lambda x: summarize_branch_chain.invoke(x),
        id=lambda x: x["id"],
        metadata=lambda x: x["metadata"],
    ) | RunnableLambda(lambda x: Document(**x))
)

intent_summary_chain = (
    split_chunk_chain |
    RunnableLambda(lambda chunks: intent_summary_doc_chain.batch(chunks))
)

### Retrieval and QA chains

In [ ]:
def get_mrr(x: dict):
    docs = x["docs"]
    doc_seq_nums = [doc.metadata["seq_num"] for doc in docs]
    print(doc_seq_nums)
    uniq_seq_nums = list(set(doc_seq_nums))
    rr_sums = dict([(seq_num, 0.0) for seq_num in uniq_seq_nums])
    counts = dict([(seq_num, 0) for seq_num in uniq_seq_nums])
    for rank, seq_num in enumerate(doc_seq_nums, 1):
        rr_sums[seq_num] = rr_sums[seq_num] + 1./rank
        counts[seq_num] += 1
    mrr = dict([(seq_num, rr_sums[seq_num]/8.) for seq_num in uniq_seq_nums])
    return mrr

In [ ]:
expected_query_retrieval_chain = (
    input_chain |
    RunnableParallel(
        docs=lambda question: expected_query_vector_db.similarity_search(
            question,
            k=8
        )
    ) |
    RunnableLambda(get_mrr)
)

In [ ]:
intent_summary_retrieval_chain = (
    input_chain |
    RunnableParallel(
        docs=lambda question: intent_summary_vector_db.similarity_search(
            question,
            k=8
        )
    ) |
    RunnableLambda(get_mrr)
)

In [ ]:
bm25_retrieval_chain = (
    input_chain |
    keyword_extract_chain |
    RunnableParallel(
        docs=lambda keywords: bm25_retriever.invoke(re.sub("[-,\s]+", " ", keywords), k=4)
    ) |
    RunnableLambda(get_mrr)
)

In [ ]:
mrr_weight = {
    "mrr_expected_query": 0.3,
    "mrr_intent_summary": 0.4,
    "mrr_bm25": 0.3,
}
def weighted_mrr(mrr_doc_scores: dict):
    mrr_names = mrr_doc_scores.keys()
    seq_num_set = set()
    for name in mrr_names:
        seq_num_set.update(mrr_doc_scores[name].keys())
    weighted_mrr = dict([(seq_num, 0.0) for seq_num in seq_num_set])

    for name in mrr_names:
        for seq_num, score in mrr_doc_scores[name].items():
            weighted_mrr[seq_num] = weighted_mrr[seq_num] + mrr_weight[name] * score
    return weighted_mrr

retrieval_chain = (
    RunnablePassthrough() |
    RunnableParallel(
        mrr_expected_query=lambda x: expected_query_retrieval_chain.invoke(x),
        mrr_intent_summary=lambda x: intent_summary_retrieval_chain.invoke(x),
        mrr_bm25=lambda x: bm25_retrieval_chain.invoke(x)
    ) | 
    # RunnableLambda(weighted_mrr)
    RunnableParallel(
        mrr_expected_query=lambda x: x["mrr_expected_query"],
        mrr_intent_summary=lambda x: x["mrr_intent_summary"],
        mrr_bm25=lambda x: x["mrr_bm25"],
        weighted_mrr=lambda x: weighted_mrr(x)
    )
)

In [ ]:
qa_chain = PromptTemplate.from_template(
    """
    내용: {context}
    질문: {question} 짧게 답변만 대답해줘
    답변: 
    """
) | llm
def get_top_context(mrr_scores: dict) -> str:
    top_k_docs = sorted(mrr_scores.items(), key=lambda x: x[1], reverse=True)[0][:2]
    documents: List[Document] = list()
    for top_doc in top_k_docs:
        documents.extend(chunk_vector_db.similarity_search("", k=10, filter={"seq_num": top_doc}))
    context = "\n".join([d.page_content for d in documents])
    return context

retrieval_and_generation_chain = (
    input_chain |
    RunnablePassthrough.assign(mrr_scores=retrieval_chain) |
    # RunnableLambda(retrieval_chain) |
    RunnableParallel(
        context=lambda x: RunnableLambda(lambda x: x["mrr_scores"]["weighted_mrr"]) | RunnableLambda(get_top_context),
        question=lambda x: x["question"],
        mrr_scores=lambda x: x["mrr_scores"]
    ) |
    RunnableParallel(
        answer=lambda x: qa_chain.invoke({"context": x["context"], "question": x["question"]}),
        mrr_scores=lambda x: x["mrr_scores"]
    )
)

## Semantic Indexing

In [465]:
documents = loader.load()

### Chunk Vector DB

In [466]:
chunk_splits = text_chunk_splitter.split_documents(documents)
chunk_vector_db = FAISS.from_documents(documents=chunk_splits, embedding=embeddings)

In [467]:
if update_vector_index["chunk"]:
    chunk_vector_db.save_local(chunk_vector_index_dirpath, "vectors")

### Expected Query Vector DB

In [468]:
if update_vector_index["expected_query"]:
    index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))
    expected_query_vector_db = FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={}
    )

    for doc in loader.lazy_load():
        batch_result = expected_query_chain.invoke(doc.page_content)
        batch_docs = [
            Document(
                page_content=query,
                metadata={"seq_num": doc.metadata["seq_num"], "expanded_query_seq_num": query_seq})
                for query_seq, query in enumerate(batch_result["expected_queries"]
            )
        ]
        expected_query_vector_db.add_documents(batch_docs)

    expected_query_vector_db.save_local(expected_query_vector_index_dirpath)
else:
    expected_query_vector_db = FAISS.load_local(expected_query_vector_index_dirpath, embeddings=embeddings, allow_dangerous_deserialization=True)

### Intent-based Summary Vector DB

In [469]:
if update_vector_index["intent_summary"]:
    index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))
    intent_summary_vector_db = FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={}
    )
    for doc in loader.lazy_load():
        batch_docs = intent_summary_chain.invoke({"question": doc.page_content, "seq_num": doc.metadata["seq_num"]})
        intent_summary_vector_db.add_documents(batch_docs)

    intent_summary_vector_db.save_local(intent_summary_vector_index_dirpath)
else:
    intent_summary_vector_db = FAISS.load_local(intent_summary_vector_index_dirpath, embeddings=embeddings, allow_dangerous_deserialization=True)

## Retrieve and QA Pipeline

In [470]:
intent_chain.invoke({"question": question})

'대응'

In [471]:
(input_chain | preproc_chain).invoke({"question": question})

{'intent': '대응',
 'sentence_type': '질문형',
 'keywords': '연구, 미세플라스틱, 햇빛, 오염줄질, 미국대학교'}

### Document Retrieval via Expected Queries Vector

In [472]:
docs = expected_query_vector_db.similarity_search_with_score(
    "유엔환경계획 조약 관련 질문",
    k=8,
    # filter={"seq_num":1}
)
# pretty_print_docs(docs)
print(f'\n{"="*256}\n\n'.join([f'[{doc[0].metadata}] {doc[0].page_content}' for doc in docs]))

[{'seq_num': 1, 'expanded_query_seq_num': 2}] 연구 결과에 따르면, 미세플라스틱은 어떤 요인에 노출되면 표면 입자가 변형되어 오염 물질을 더 잘 흡수하는 상태로 변형되는 거래?

[{'seq_num': 2, 'expanded_query_seq_num': 3}] 이 조약이 작성된 배경은 무엇이며, 왜 작성되었다고 하나요?

[{'seq_num': 4, 'expanded_query_seq_num': 0}] 플라스틱병에 든 생수에 미세플라스틱이 많이 함유되어 있다는 연구결과가 어떻게 발표되었나요?

[{'seq_num': 4, 'expanded_query_seq_num': 2}] 먹는물네트워크와 대한환경공학회가 주최한 ‘생수와 미세플라스틱, 안전한 먹는 물을 위한 공동 노력’ 포럼이 열린 곳과 날짜는 언제였나요?

[{'seq_num': 2, 'expanded_query_seq_num': 7}] 조약에 따르면, 각 국가가 수행해야 하는 플라스틱 관련 의무는 무엇이죠?

[{'seq_num': 1, 'expanded_query_seq_num': 1}] 미국 애리조나 주립대학의 연구에서 확인된 미세플라스틱의 유해화학물질 흡수 증가 현상에 대해 자세히 말해줘

[{'seq_num': 2, 'expanded_query_seq_num': 4}] 어떤 기업이 플라스틱 생산 및 사용에 영향을 받게 될까요?

[{'seq_num': 4, 'expanded_query_seq_num': 4}] 호주 연구팀에 따르면 인간이 매주 얼마나 많은 미세 플라스틱을 섭취하는 것으로 추산되는가요?


In [522]:
    # "국제사회의 플라스틱 오염 인식",
question = "화학물질이나 고분자 수입자 규정"
question = "유엔환경계획 조약 관련 질문"
question = "미세플라스틱 햇빛 노출 연구 대학"
question = "정부의 미세플라스틱 정책 부족 비판"
question = "페트병 생수 나노 플라스틱 검출 수"

expected_query_retrieval_chain.invoke(question)

[3, 4, 2, 3, 2, 4, 3, 4]


{2: 0.06666666666666667, 3: 0.17410714285714285, 4: 0.09895833333333333}

In [477]:
intent_summary_vector_db.similarity_search_with_score(
    question,
    k=8
)

[(Document(metadata={'seq_num': 1, 'chunk_seq_num': 1}, page_content='미세플라스틱, 햇빛 등 노출로 표면 변형 후 유해화학물질흡수 증가(애리조나 주립대학 연구 결과)'),
  21431.03),
 (Document(metadata={'seq_num': 4, 'chunk_seq_num': 4}, page_content='생수, 미세플라스틱 함유, 위해성 평가 필요(2017년 환경부 조사 당시 생수 1개 제품에서 1개 입자 검출된 바 있음)'),
  21858.494),
 (Document(metadata={'seq_num': 2, 'chunk_seq_num': 0}, page_content='유엔환경계획은 글로벌 조약 초안에서 플라스틱 오염을 줄이기 위해 우려되는 화학물질 및 고분자 사용을 금지하고, 국가정책기관에 의한 제도적 변화를 통해 기업에 영향을 미치도록 함.'),
  22774.396),
 (Document(metadata={'seq_num': 3, 'chunk_seq_num': 4}, page_content='- 태평양 해양쓰레기 청소 시 장벽 발생: 출입국 비자 문제, 관세, 항구 접근 제한 등\n- 하천 쓰레기 청소 미흡: 플라스틱 유입 통로인 하천에 대한 대처 부족\n- 2040년 미세플라스틱 증가 예상: 현재의 2.6배까지 증가 가능성\n- 해양쓰레기 영향 심각: 해양생태계, 관련 산업 등에 악영향 지속'),
  23569.344),
 (Document(metadata={'seq_num': 2, 'chunk_seq_num': 7}, page_content='유엔환경계획은 전 세계적으로 플라스틱 오염을 줄이기 위한 새로운 글로벌 조약 초안*을 발표함.'),
  24181.93),
 (Document(metadata={'seq_num': 4, 'chunk_seq_num': 1}, page_content='- 2017년 환경부 조사: 생수 한 개 제품에서 1개 입자 검출\n- 지난해 노르웨이, 

In [526]:
    # "국제사회의 플라스틱 오염 인식",
question = "유엔환경계획 조약 관련 질문"
question = "페트병 생수 나노 플라스틱 검출 수"
question = "정부의 미세플라스틱 정책 부족 비판"
question = "화학물질이나 고분자 수입자 규정"
question = "미세플라스틱 햇빛 노출 연구 대학"

intent_summary_retrieval_chain.invoke(question)

[1, 1, 1, 4, 1, 4, 3, 2]


{1: 0.25416666666666665,
 2: 0.015625,
 3: 0.017857142857142856,
 4: 0.05208333333333333}

In [481]:
bm25_retrieval_chain.invoke(question)

[4, 1, 2, 3]


{1: 0.0625, 2: 0.041666666666666664, 3: 0.03125, 4: 0.125}

### Generate Answer with Retrieved Documents using Weighted AVG MRR

In [521]:
question = "매주 1인당 섭취하는 플라스틱 양은?"
question = "1인당 섭취하는 플라스틱 양은?"
question = "미세플라스틱 오염에 우려가 커진다고 말한 사람은 누구야?"
# question = "시중에서 판매되는 4개 브랜드의 페트병 생수 미세플라스틱을 검출한 나라들은 어디인가요?"
question = '미세플라스틱의 위해성 평가와 정부 차원의 대응에 대한 목소리에서 특히 중요한 부분은 무엇인가요?'
question = "한사람이 매주 먹는 미세플라스틱 양"

retrieval_and_generation_chain.invoke(question)

[3, 1, 4, 4, 3, 2, 4, 2]
[1, 4, 4, 3, 2, 4, 1, 3]
[4, 3, 2, 1]


{'answer': AIMessage(content='매주 먹는 미세플라스틱의 양은 평균적으로 약 0.1~0.2 g 정도입니다.\n\n', response_metadata={'model': 'tiger-gemma2', 'created_at': '2024-08-09T04:01:03.557274871Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1256748342, 'load_duration': 29211560, 'prompt_eval_count': 606, 'prompt_eval_duration': 212304000, 'eval_count': 32, 'eval_duration': 961059000}, id='run-10cbb36f-2239-467a-b053-6cedfcefadec-0', usage_metadata={'input_tokens': 606, 'output_tokens': 32, 'total_tokens': 638}),
 'mrr_scores': {'mrr_expected_query': {1: 0.0625,
   2: 0.03645833333333333,
   3: 0.15,
   4: 0.0907738095238095},
  'mrr_intent_summary': {1: 0.14285714285714285,
   2: 0.025,
   3: 0.046875,
   4: 0.12499999999999999},
  'mrr_bm25': {1: 0.03125, 2: 0.041666666666666664, 3: 0.0625, 4: 0.125},
  'weighted_mrr': {1: 0.08526785714285713,
   2: 0.033437499999999995,
   3: 0.0825,
   4: 0.11473214285714284}}}

In [501]:
expected_query_vector_db.docstore.__dict__

{'_dict': {'1eb71158-c05e-4cbe-9e76-d91e59207d98': Document(metadata={'seq_num': 1, 'expanded_query_seq_num': 0}, page_content='미세플라스틱의 노후화 과정에서 유해 화학 물질 흡수가 증가한 점을 설명해줘'),
  'c6804b18-bd81-4195-8163-1c6b8e3b1c6e': Document(metadata={'seq_num': 1, 'expanded_query_seq_num': 1}, page_content='미국 애리조나 주립대학의 연구에서 확인된 미세플라스틱의 유해화학물질 흡수 증가 현상에 대해 자세히 말해줘'),
  '2311e930-e854-4e43-91f5-779e4600990c': Document(metadata={'seq_num': 1, 'expanded_query_seq_num': 2}, page_content='연구 결과에 따르면, 미세플라스틱은 어떤 요인에 노출되면 표면 입자가 변형되어 오염 물질을 더 잘 흡수하는 상태로 변형되는 거래?'),
  '80a28bec-a586-4037-b723-079f445fbe05': Document(metadata={'seq_num': 1, 'expanded_query_seq_num': 3}, page_content='연구팀이 사용한 인위적으로 생성된 미세플라스틱의 종류는 무엇이며, 이를 풍화시키기 위해 어떤 방법을 사용했나요?'),
  '727c8fd9-2028-4678-a1a6-7ec5426ecf5f': Document(metadata={'seq_num': 1, 'expanded_query_seq_num': 4}, page_content='풍화 과정에서 오염 물질 모델인 페난트렌과 메틸렌 블루에 노출된 후에 일어난 현상이 무엇입니까?'),
  '9d2af67e-227e-405c-9a32-deb4c0757a3f': Document(metadata={'seq_num': 1, 'expanded_query

In [503]:
question

'미세플라스틱 오염에 우려가 커진다고 말한 사람은 누구야?'

In [502]:
expected_query_vector_db.similarity_search_with_score(
    question,
    k=8
)

[(Document(metadata={'seq_num': 2, 'expanded_query_seq_num': 2}, page_content='UNEP의 목표는 언제까지 조약을 완성하는 것입니다.'),
  13541.711),
 (Document(metadata={'seq_num': 2, 'expanded_query_seq_num': 1}, page_content='‘제로 초안(zero draft)’이라는 용어는 무슨 의미를 가지나요?'),
  13581.148),
 (Document(metadata={'seq_num': 1, 'expanded_query_seq_num': 4}, page_content='풍화 과정에서 오염 물질 모델인 페난트렌과 메틸렌 블루에 노출된 후에 일어난 현상이 무엇입니까?'),
  13625.66),
 (Document(metadata={'seq_num': 2, 'expanded_query_seq_num': 3}, page_content='이 조약이 작성된 배경은 무엇이며, 왜 작성되었다고 하나요?'),
  13778.119),
 (Document(metadata={'seq_num': 4, 'expanded_query_seq_num': 0}, page_content='플라스틱병에 든 생수에 미세플라스틱이 많이 함유되어 있다는 연구결과가 어떻게 발표되었나요?'),
  14004.192),
 (Document(metadata={'seq_num': 4, 'expanded_query_seq_num': 1}, page_content='미세플라스틱의 위해성 평가와 정부 차원의 대응에 대한 목소리에서 특히 중요한 부분은 무엇인가요?'),
  14834.953),
 (Document(metadata={'seq_num': 3, 'expanded_query_seq_num': 2}, page_content='해양쓰레기 청소 과정에서 발생하는 문제는 무엇인가요?'),
  14877.024),
 (Document(metadata={'seq_num': 2, '

In [428]:
chunk_vector_db.similarity_search(
    question, filter={"seq_num": 3}, k=10
)

[Document(metadata={'source': '/mnt/d/temp/user/ed/mart/mmplastic/20240725_154000_000000/data.json', 'seq_num': 3, 'title': 'APEC 국가간 무역장벽, 태평양 해양쓰레기 청소 방해', 'date': '2022-12-18'}, page_content='태평양의 플라스틱 등 해양쓰레기 청소가 국가 간 무역장벽의 방해를 받고 있다고 아시아태평양경제협력체(APEC)가 진단했다.'),
 Document(metadata={'source': '/mnt/d/temp/user/ed/mart/mmplastic/20240725_154000_000000/data.json', 'seq_num': 3, 'title': 'APEC 국가간 무역장벽, 태평양 해양쓰레기 청소 방해', 'date': '2022-12-18'}, page_content='또 해변 쓰레기 청소는 수십 년간 진행돼 왔으나, 플라스틱 쓰레기 등이 바다로 유입되는 통로인 하천에 대한 청소는 제대로 이뤄지지 않고 있다고 지적했다.\n\n결국 이들 미세 플라스틱 부유물은 태평양 3대 거대 쓰레기 지역으로 모인 뒤 바람과 해류들로 인해 잘게 분쇄돼 해저로 침전된다.'),
 Document(metadata={'source': '/mnt/d/temp/user/ed/mart/mmplastic/20240725_154000_000000/data.json', 'seq_num': 3, 'title': 'APEC 국가간 무역장벽, 태평양 해양쓰레기 청소 방해', 'date': '2022-12-18'}, page_content='이처럼 해저에 가라앉는 미세플라스틱의 규모가 2040년에는 현재의 2.6배나 될 것으로 우려된다.\n\n이들 플라스틱 쓰레기가 해상과 해변 생태계, 이들 생태계에 의존하는 산업계에 미치는 영향을 가늠하기조차 쉽지 않다고 APEC은 지적했다.'),
 Document(metadata={'source': '/mnt/d/tem

In [429]:
chunk_vector_db.search(
    question, filter={"seq_num": 3}, search_type="similarity", k=10
)

[Document(metadata={'source': '/mnt/d/temp/user/ed/mart/mmplastic/20240725_154000_000000/data.json', 'seq_num': 3, 'title': 'APEC 국가간 무역장벽, 태평양 해양쓰레기 청소 방해', 'date': '2022-12-18'}, page_content='태평양의 플라스틱 등 해양쓰레기 청소가 국가 간 무역장벽의 방해를 받고 있다고 아시아태평양경제협력체(APEC)가 진단했다.'),
 Document(metadata={'source': '/mnt/d/temp/user/ed/mart/mmplastic/20240725_154000_000000/data.json', 'seq_num': 3, 'title': 'APEC 국가간 무역장벽, 태평양 해양쓰레기 청소 방해', 'date': '2022-12-18'}, page_content='또 해변 쓰레기 청소는 수십 년간 진행돼 왔으나, 플라스틱 쓰레기 등이 바다로 유입되는 통로인 하천에 대한 청소는 제대로 이뤄지지 않고 있다고 지적했다.\n\n결국 이들 미세 플라스틱 부유물은 태평양 3대 거대 쓰레기 지역으로 모인 뒤 바람과 해류들로 인해 잘게 분쇄돼 해저로 침전된다.'),
 Document(metadata={'source': '/mnt/d/temp/user/ed/mart/mmplastic/20240725_154000_000000/data.json', 'seq_num': 3, 'title': 'APEC 국가간 무역장벽, 태평양 해양쓰레기 청소 방해', 'date': '2022-12-18'}, page_content='이처럼 해저에 가라앉는 미세플라스틱의 규모가 2040년에는 현재의 2.6배나 될 것으로 우려된다.\n\n이들 플라스틱 쓰레기가 해상과 해변 생태계, 이들 생태계에 의존하는 산업계에 미치는 영향을 가늠하기조차 쉽지 않다고 APEC은 지적했다.'),
 Document(metadata={'source': '/mnt/d/tem

In [430]:
chunk_vector_db.search(
    question, filter={"seq_num": 3}, search_type="mmr", k=10
)

[Document(metadata={'source': '/mnt/d/temp/user/ed/mart/mmplastic/20240725_154000_000000/data.json', 'seq_num': 3, 'title': 'APEC 국가간 무역장벽, 태평양 해양쓰레기 청소 방해', 'date': '2022-12-18'}, page_content='태평양의 플라스틱 등 해양쓰레기 청소가 국가 간 무역장벽의 방해를 받고 있다고 아시아태평양경제협력체(APEC)가 진단했다.'),
 Document(metadata={'source': '/mnt/d/temp/user/ed/mart/mmplastic/20240725_154000_000000/data.json', 'seq_num': 3, 'title': 'APEC 국가간 무역장벽, 태평양 해양쓰레기 청소 방해', 'date': '2022-12-18'}, page_content='다만 APEC의 2020년 연구에 따르면 해상쓰레기가 다양한 산업계에 미치는 직접적인 영향이 최근 몇 년간 크게 증가했다.\n\n과거 연구에서는 이들 쓰레기가 관광과 교통, 양식산업 등에 미치는 피해 규모가 109억달러(약 14조3천억원)에 달하는 것으로 추산됐다.'),
 Document(metadata={'source': '/mnt/d/temp/user/ed/mart/mmplastic/20240725_154000_000000/data.json', 'seq_num': 3, 'title': 'APEC 국가간 무역장벽, 태평양 해양쓰레기 청소 방해', 'date': '2022-12-18'}, page_content='또 해변 쓰레기 청소는 수십 년간 진행돼 왔으나, 플라스틱 쓰레기 등이 바다로 유입되는 통로인 하천에 대한 청소는 제대로 이뤄지지 않고 있다고 지적했다.\n\n결국 이들 미세 플라스틱 부유물은 태평양 3대 거대 쓰레기 지역으로 모인 뒤 바람과 해류들로 인해 잘게 분쇄돼 해저로 침전된다.'),
 Document(metadata={'source

In [431]:
intent_summary_vector_db.similarity_search(
    question, filter={"seq_num": 3}
)

[Document(metadata={'seq_num': 3, 'chunk_seq_num': 3}, page_content='- 태평양 해양쓰레기 청소 활동은 출입국 비자 문제, 관세, 항구 접근 제한 등 무역 장벽으로 어려움을 겪고 있다(2023년 1월 15일).\n\n- 플라스틱 쓰레기는 하천에서 바다로 유입되는 과정이 미흡하고, 해저에 가라앉아 문제가 심각해지고 있다.'),
 Document(metadata={'seq_num': 3, 'chunk_seq_num': 0}, page_content='APEC은 태평양 해양쓰레기 청소를 방해하는 무역장벽과 하천 청소 부재, 미세 플라스틱의 증가 등을 지적했습니다.'),
 Document(metadata={'seq_num': 3, 'chunk_seq_num': 1}, page_content='APEC: 태평양 플라스틱 쓰레기 청소, 국가 간 무역장벽 방해 (2023년 현재) 해상 쓰레기 영향: 관광, 교통, 양식산업 등 다양한 산업계에 미치는 직접적 영향 증가')]

In [432]:
expected_query_vector_db.similarity_search(
    question,
    filter={"seq_num": 4},
    fetck_k=10
)

[Document(metadata={'seq_num': 4, 'expanded_query_seq_num': 2}, page_content='먹는물네트워크와 대한환경공학회가 주최한 ‘생수와 미세플라스틱, 안전한 먹는 물을 위한 공동 노력’ 포럼이 열린 곳과 날짜는 언제였나요?'),
 Document(metadata={'seq_num': 4, 'expanded_query_seq_num': 5}, page_content='보통 미세플라스틱은 무엇을 말하며, 나노 플라스틱은 어떤 기준으로 분류되나요?'),
 Document(metadata={'seq_num': 4, 'expanded_query_seq_num': 0}, page_content='플라스틱병에 든 생수에 미세플라스틱이 많이 함유되어 있다는 연구결과가 어떻게 발표되었나요?'),
 Document(metadata={'seq_num': 4, 'expanded_query_seq_num': 1}, page_content='미세플라스틱의 위해성 평가와 정부 차원의 대응에 대한 목소리에서 특히 중요한 부분은 무엇인가요?')]

In [433]:
chunk_vector_db.similarity_search(
    question,
    filter={"seq_num": 4},
    fetck_k=10
)

[Document(metadata={'source': '/mnt/d/temp/user/ed/mart/mmplastic/20240725_154000_000000/data.json', 'seq_num': 4, 'title': '생수는 과연 안전할까?···건강 위협하는 미세 플라스틱', 'date': '2024-07-31'}, page_content='플라스틱병에 든 생수에 미세플라스틱이 다량 함유되어있다는 연구결과가 잇따라 발표되면서 제대로 된 위해성 평가와 정부 차원의 정책적 대응이 필요하다는 목소리가 나왔다.'),
 Document(metadata={'source': '/mnt/d/temp/user/ed/mart/mmplastic/20240725_154000_000000/data.json', 'seq_num': 4, 'title': '생수는 과연 안전할까?···건강 위협하는 미세 플라스틱', 'date': '2024-07-31'}, page_content='이날 토론자로 나선 백명수 시민환경연구소 소장은 "미세플라스틱 오염에 대한 우려가 커지고 있지만, 정부 정책은 우려를 불식시키지 못하고 있다"고 비판했다.'),
 Document(metadata={'source': '/mnt/d/temp/user/ed/mart/mmplastic/20240725_154000_000000/data.json', 'seq_num': 4, 'title': '생수는 과연 안전할까?···건강 위협하는 미세 플라스틱', 'date': '2024-07-31'}, page_content='보통 미세플라스틱은 5㎜ 이하의 작은 플라스틱 조각을 말한다. 1㎛ 이하는 나노 플라스틱으로 분류된다. 인간이 만들어낸 물건에서 떨어져 나간 이 작은 플라스틱은 대기, 담수, 해양, 지하수, 토양 등 모든 곳에 흩뿌려져 지구 전체를 순환한다.'),
 Document(metadata={'source': '/mnt/d/temp/user/ed/mart/mmplastic/20240725_154000_000000/d

In [434]:
docs = expected_query_vector_db.search("페난트렌과 메틸렌 블루", search_type="mmr")
print('\n'.join([f"[{doc.metadata}]{doc.page_content}" for doc in docs]))

[{'seq_num': 1, 'expanded_query_seq_num': 2}]연구 결과에 따르면, 미세플라스틱은 어떤 요인에 노출되면 표면 입자가 변형되어 오염 물질을 더 잘 흡수하는 상태로 변형되는 거래?
[{'seq_num': 1, 'expanded_query_seq_num': 7}]실제 환경에서 수십 년 동안 풍화된 미세플라스
[{'seq_num': 1, 'expanded_query_seq_num': 1}]미국 애리조나 주립대학의 연구에서 확인된 미세플라스틱의 유해화학물질 흡수 증가 현상에 대해 자세히 말해줘
[{'seq_num': 2, 'expanded_query_seq_num': 2}]UNEP의 목표는 언제까지 조약을 완성하는 것입니다.


## Serving

#### Test Question Classification
- for SentenceType / Intent Classification

In [435]:
sent_type_chain.invoke(question)

'질문형'

In [436]:
question = """
미세플라스틱(Microplastic)이 환경에서 노후화가 진행됨에 따라 더 많은 유해화학물질을 흡수하는 것으로 밝혀졌다. 이는 미국 애리조나 주립대학이 수행한 연구에 따라 확인되었으며, 연구 결과에 따르면 미세플라스틱은 햇빛 등에 노출 시 표면입자가 변형되어 유기오염물질을 더 잘 흡수할 수 있는 상태로 변형되는 것으로 밝혀졌다.\n\n연구팀은 폴리에틸렌(Polyethylene) 및 폴리프로필렌(Polypropylene)을 사용하여 미세플라스틱을 인위적으로 생성하고, 자외선 및 과산화수소를 이용해 풍화시킨 후, 풍화된 미세플라스틱 입자를 오염물질 모델인 페난트렌(phenanthrene)및 메틸렌 블루(methylene blue)에 노출시켰다.
"""
(input_chain | question_classify_chain).invoke(question)

{'sentence_type': '서술형',
 'intent': '영향',
 'question': '\n미세플라스틱(Microplastic)이 환경에서 노후화가 진행됨에 따라 더 많은 유해화학물질을 흡수하는 것으로 밝혀졌다. 이는 미국 애리조나 주립대학이 수행한 연구에 따라 확인되었으며, 연구 결과에 따르면 미세플라스틱은 햇빛 등에 노출 시 표면입자가 변형되어 유기오염물질을 더 잘 흡수할 수 있는 상태로 변형되는 것으로 밝혀졌다.\n\n연구팀은 폴리에틸렌(Polyethylene) 및 폴리프로필렌(Polypropylene)을 사용하여 미세플라스틱을 인위적으로 생성하고, 자외선 및 과산화수소를 이용해 풍화시킨 후, 풍화된 미세플라스틱 입자를 오염물질 모델인 페난트렌(phenanthrene)및 메틸렌 블루(methylene blue)에 노출시켰다.\n'}

# Garage

In [437]:
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[overlapped_sentences_splitter, redundant_filter, relevant_filter]
    # transformers=[redundant_filter, relevant_filter]
)

In [438]:
chunk_retriever = chunk_vector_db.as_retriever(
    search_kwargs={"k": 10}
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, base_retriever=chunk_retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    # question
    # "태평양 쓰레기 청소가 무역방벽 유발"
    "생수 미세플라스틱 연구자 찾아줘"
)
# compressed_docs = compression_retriever.invoke(
#     question
# )
pretty_print_docs_with_similarity(compressed_docs)

RANK:1
SEQ_NUM:[1], SIM[0.9049526960338856]
미세플라스틱, 노후화 될수록 더 많은 오염물질 흡착:

평가하기 위해서는 표면 풍화와 오염물질 흡착에 대한 복잡한 상호작용을 이해하는 것이 매우 중요하다.‘‘고 결론지었다.
----------------------------------------------------------------------------------------------------
RANK:2
SEQ_NUM:[2], SIM[0.9020680308094472]
UN, 플라스틱 내 우려물질 사용 금지를 위한 ‘플라스틱 조약‘ 초안 발표:

조약 초안에 따르면, 조약 당사국은 미세플라스틱을 포함한 유해화학물질이 대기, 토양, 수질 및 생태계로 배출되는 것을 ‘방지 및 제거‘ 해야 한다고 명시하고 있으며, 이를 위해 아래 3 가지 방안을 제시함.
----------------------------------------------------------------------------------------------------
RANK:3
SEQ_NUM:[2], SIM[0.892892688079204]
UN, 플라스틱 내 우려물질 사용 금지를 위한 ‘플라스틱 조약‘ 초안 발표:

- 쉽게 재활용할 수 없는 고분자 및 브롬계난연제를 포함하여 안전하고 품질 높은 2차 재료의 재활용성 또는 순환성을 저해하는 물질

- 미세플라스틱과 같이 환경에서 잘 분해되지 않아 방출했을 때 위험성이 있는 물질
----------------------------------------------------------------------------------------------------
RANK:4
SEQ_NUM:[3], SIM[0.8806877119130335]
APEC 국가간 무역장벽, 태평양 해양쓰레기 청소 방해:

결국 이들 미세 플라스틱 부유물은 태평양 3대 거대 쓰레기 지역으로 모인 뒤 바람과 해류들로 인해 잘게 분쇄돼 해저로 침전